In [11]:
import os

import numpy as np
import tensorflow as tf
from imutils import paths
from tensorflow.data import AUTOTUNE
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from pyimagesearch.helpers import benchmark

In [24]:
def load_images(imagePath):
    """
    read the image from disk, decode it, resize it,
    and scale the pixels intensities to the range [0, 1]

    :param imagePath:
    :return:
    """
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, (96, 96)) / 255.0

    # grab the label and encode it
    label = tf.strings.split(imagePath, os.path.sep)[-2]
    oneHot = label == classNames
    encodedLabel = tf.argmax(oneHot)

    # return the image and the integer encoded label
    return image, encodedLabel

In [13]:
data_dir = 'fruits'

# initialize batch size and number of steps
BS = 64
NUM_STEPS = 1000

In [14]:
# grab the list of images in our dataset directory and grab all unique class names
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(data_dir))
classNames = np.array(sorted(os.listdir(data_dir)))

[INFO] loading image paths...


In [15]:
# create a standard image generator object
print("[INFO] creating a ImageDataGenerator object...")
imageGen = ImageDataGenerator(rescale=1.0 / 255)
dataGen = imageGen.flow_from_directory(
    data_dir,
    target_size=(96, 96),
    batch_size=BS,
    class_mode="categorical",
    color_mode="rgb"
)

[INFO] creating a ImageDataGenerator object...
Found 6688 images belonging to 7 classes.


In [16]:
# '''
# benchmark the image data generator and display the number of data points generated, 
# along with the time taken to perform the operation
# '''
# totalTime = benchmark(dataGen, NUM_STEPS)
# print(f"[INFO] ImageDataGenerator generated {BS * NUM_STEPS} images in {totalTime:.2f} seconds...")

In [17]:
# build the dataset and data input pipeline
print("[INFO] creating a tf.data input pipeline..")
dataset = tf.data.Dataset.from_tensor_slices(imagePaths)
dataset = dataset.shuffle(1024).map(load_images, num_parallel_calls=AUTOTUNE).cache().repeat().batch(BS).prefetch(AUTOTUNE)

[INFO] creating a tf.data input pipeline..


2022-06-28 15:09:46.796699: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
'''
create a dataset iterator, benchmark the tf.data pipeline, 
and display the number of data points generated, along with the time taken
'''
datasetGen = iter(dataset)
totalTime = benchmark(datasetGen, NUM_STEPS)
print(f"[INFO] tf.data generated {BS * NUM_STEPS} images in {totalTime:.2f} seconds...")

[INFO] tf.data generated 64000 images in 35.46 seconds...


In [20]:
img_path = imagePaths[0]
img_path

'fruits/apple/apple_941.jpg'

In [25]:
x, y = load_images(img_path)

In [22]:
x

<tf.Tensor: shape=(96, 96, 3), dtype=float32, numpy=
array([[[0.00000000e+00, 3.92156886e-03, 1.96078438e-02],
        [0.00000000e+00, 0.00000000e+00, 7.84313772e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.38235301e-01, 1.64705887e-01, 1.75980389e-01],
        [1.21568628e-01, 3.72549035e-02, 1.47058824e-02],
        [1.85784310e-01, 1.60784319e-01, 1.03431374e-01]],

       [[0.00000000e+00, 0.00000000e+00, 1.56862754e-02],
        [0.00000000e+00, 4.90196107e-04, 8.33333377e-03],
        [0.00000000e+00, 3.92156886e-03, 0.00000000e+00],
        ...,
        [5.62745094e-01, 6.43137276e-01, 6.82843149e-01],
        [1.34803921e-01, 9.65686291e-02, 8.77451003e-02],
        [7.20588267e-02, 1.91176478e-02, 4.41176491e-03]],

       [[0.00000000e+00, 4.90196107e-04, 8.33333377e-03],
        [0.00000000e+00, 0.00000000e+00, 7.84313772e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.95098042e-01, 2.9362744

In [23]:
y

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [37]:
def tf_read_img(img_path):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_image(image, channels=3)
    print(image)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)

    return image

In [38]:
tf_read_img(img_path)

tf.Tensor(
[[[  0   0   4]
  [  0   0   4]
  [  0   0   4]
  ...
  [ 26  22  13]
  [ 39  39  27]
  [ 54  56  42]]

 [[  0   0   4]
  [  0   0   4]
  [  0   1   5]
  ...
  [ 56  50  38]
  [ 45  43  30]
  [ 50  51  37]]

 [[  0   0   4]
  [  0   1   5]
  [  0   1   5]
  ...
  [ 33  26   7]
  [ 44  38  24]
  [ 40  36  25]]

 ...

 [[176 120  97]
  [188 134 108]
  [185 131 103]
  ...
  [ 95 141 141]
  [ 86 132 132]
  [ 73 119 117]]

 [[117  48  33]
  [129  62  45]
  [119  52  33]
  ...
  [ 82 130 132]
  [ 61 106 109]
  [ 42  88  88]]

 [[113  36  26]
  [106  32  19]
  [108  34  21]
  ...
  [ 51  99 103]
  [ 35  80  83]
  [ 32  77  80]]], shape=(360, 480, 3), dtype=uint8)


<tf.Tensor: shape=(360, 480, 3), dtype=float32, numpy=
array([[[0.        , 0.        , 0.01568628],
        [0.        , 0.        , 0.01568628],
        [0.        , 0.        , 0.01568628],
        ...,
        [0.10196079, 0.08627451, 0.0509804 ],
        [0.15294118, 0.15294118, 0.10588236],
        [0.21176472, 0.21960786, 0.16470589]],

       [[0.        , 0.        , 0.01568628],
        [0.        , 0.        , 0.01568628],
        [0.        , 0.00392157, 0.01960784],
        ...,
        [0.21960786, 0.19607845, 0.14901961],
        [0.1764706 , 0.16862746, 0.11764707],
        [0.19607845, 0.20000002, 0.14509805]],

       [[0.        , 0.        , 0.01568628],
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784],
        ...,
        [0.12941177, 0.10196079, 0.02745098],
        [0.17254902, 0.14901961, 0.09411766],
        [0.15686275, 0.14117648, 0.09803922]],

       ...,

       [[0.6901961 , 0.47058827, 0.3803922 ],
        [0.73

In [40]:
qq = tf.constant(52, dtype=tf.float32)

In [41]:
qq.numpy()

52.0